In [ ]:
import numpy as np
import pandas as pd
import time

#defining the simplex method, It can be used to solve any LP.
#it takes 5 matrices: the basic variables, indeces of the basic matrices, non-basic variables, Indeces of non-basic varibales, Objectve function coefficients, Constraints coefficients
#the constraints RHS values are not needed during the iterations, but were used for debugging.
def Simplex_Method(XB,IB,XN,IN,c,A,b):
  while True:

    m,n = A.shape                                           #storing the shape of A, which indicates the number of constraints and number of variables (including slacks and artificial variables)

    B = np.zeros((m, m))                                    #storing the current iteration B matrix based on the indeces of the basic variables, stacked column by column from matrix A
    for i in range(m):
      B[:, i] = A[:, int(IB[:,i]-1)]

    cB = np.zeros((m, 1))                                   #storing the current iteration cB column based on the indeces of the basic variables, stacked row by row from matrix c
    for i in range(m):
      cB[i, :] = c[int(IB[:,i]-1), :]

    pi = np.linalg.solve(B.T, cB)                           #computing pi values using linalg function which solves a set of linear equations instead of getting the inverse of the a matrix

    N_loc = 0                                               #initializing the location of each non-basic variable we got through
    for j in range(n-m):                                    #going through the non basic variables indeces
      cN = float(c[int(IN[:,j]-1),:])                       #getting the c value for the current non-basic variable
      NN = A[:, int(IN[:,j]-1)].reshape(m,1)                #getting the A matrix column associated with the current non-basic variable
      r = cN - (pi.T)@NN                                    #Calculating the reduced cost for the non-basic variable
      if r < 0:                                             #checking if r is negative
        Vin_enters = int(IN[:,j])                           #storing the index of the current non-basic variable with the negative r as the chosed variable to enter
        break                                               #since the indeces are sorted, no need to check for more negative r values. following bland's rule by taking the variable with the smallest index
      else:N_loc +=1                                        #updating the location

    if N_loc == (n-m):                                      #if all r were positive (aka we went over all the non basic variables without getting a negative r), we're optimal, stop iterating
      break

    d_1 = np.linalg.solve(B, -NN)                           #computing d values using linalg function
    e = np.zeros(((n-m), 1))                                #getting the e vector as all 0, same number as the non-basic variables
    e[N_loc, :] = 1                                         #assigning 1 to the location of the chosen non-basic variable to enter the basic set
    d_2 = np.vstack((d_1, e))                               #stacking d and e together (for demonstration and debuging only, was not used in the code)

    #initializing Alpha, instead of using a random value, so we can start comparing the alpha values and get the smallest one
    alpha_index = 0
    for d_trial in d_1:                                     #going through the values of d - basic
      d_trial = float(d_trial)
      if d_trial < 0:
        alpha = (-1*(float(XB[alpha_index]))/d_trial)+1     #initializing alpha as the first value that we got + 1
        break
      alpha_index += 1

    #checking boundedness and calculating alpha
    d_loc = 0                                               #initializing the location variable
    positive_d = 0                                          #initialising the number of positive d
    for d in d_1:                                           #going throuhg the d values
      d = float(d)
      if d < 0:                                             #checking if d is negative
        alpha_New = (-1*(float(XB[d_loc])))/d               #computing Alpha for each d value < 0
        if alpha_New < alpha:                               #updating alpha to the new smaller alpha
          alpha = alpha_New
          Vin_exits = int(IB[:, d_loc])                     #storing the indix of the variable with the smallest alpha to be removed from the basic variable set
          V_exits_loc = d_loc                               #storing the location of the leaving variable
        d_loc += 1                                          #updating the location
      else:
        positive_d +=1                                      #updating the number of positive d
        d_loc += 1                                          #updating the location

    if positive_d == d_loc:                                 #checking if all d is positive (unbounded)
      XB = 'LP Unbounded'
      XN = 'LP Unbounded'
      IB = 'LP Unbounded'
      IN = 'LP Unbounded'
      break

    XB_new = XB + (alpha * d_1)                             #calculating the new XB
    X_exits = XB_new[V_exits_loc]                           #storing the value of the variable that left (for demonstration and debuging only, was not used in the code since it's 0)

    XN_new = XN + (alpha * e)                               #calculating the new XN
    X_enters = float(XN_new[N_loc])                         #storing the value of the variable that entered

    XB = XB_new
    XB[V_exits_loc] = X_enters                              #replacing the value of the variable that left with the one that entered
    IB[0,V_exits_loc] = Vin_enters                          #replacing the index of the variable that left with the one that entered
    XB = XB[np.argsort(IB)].reshape(m,1)                    #sorting XB to get in order based on the indices in IB
    IB = np.sort(IB)                                        #sorting IB to get in order

    XN = XN_new
    XN[N_loc] = 0                                           #replacing the value of the variable that entered with 0
    IN[0,N_loc] = Vin_exits                                 #replacing the index of the variable that entered with the one that left
    IN = np.sort(IN)                                        #sorting IN to get in order

  return XB, IB, XN, IN

In [ ]:
def BIG_M(A,b,c):
  #defining the Big M method since our problem needs artificial variables to initial a BFS.
  #it needs Objectve function coefficients, Constraints coefficients, and constraints RHS values.

  m,n = A.shape                                             #storing the shape of A, which indicates the number of constraints and number of variables

  cb_1 = np.ones((m, 1))*1000000                            #creating the objective function coefficient to the new artificial variables (BIG M), as much as we have constaints
  cn_1 = c
  c_1 = np.vstack((cn_1, cb_1))                             #stacking the original c with the BIG M
  XB_1 = b                                                  #giving the artificial variables the value of b as an initial basic variables
  XN_1 = np.zeros(((n), 1))                                 #giving the rest of the variables a value of 0 as the non-basic variables

  IB_1 = np.zeros((1, m))
  for i in range(m):
    IB_1[:, i] = i+n+1                                      #storing the basic variables indeces, starting from the number of the last original variable and increasing as much as we have constaints

  IN_1 = np.zeros((1, n))
  for i in range(n):
    IN_1[:, i] = i+1                                        #storing the non-basic variables indeces, as much as the original variables.

  identity = np.eye(m)
  A_1 = np.hstack((A, identity))                            #adding an identity matrix with the size of constraints as the A coeffecient of the artificial variables

  XB, IB, XN, IN = Simplex_Method(XB_1,IB_1,XN_1,IN_1,c_1,A_1,b)          #calling the simplex method fucntion to  solve our LP

  for IB_index in range(IB.size):
    if int(IB[:,IB_index]) > n:
      if XB[IB_index,0] == 0:
        for IN_index in range(IN.size):
          if int(IN[:,IN_index]) <= n:
            IB[:,IB_index] = int(IN[:,IN_index])            #if we have an artificial variable with a 0 value in our basic variables, it shall be replaced with a non-basic variable
            IN = np.delete(IN, IN_index, axis=1)
            XN = np.delete(XN, IN_index, axis=0)
            break
      else:                                                 #if we have an artificial variable with a non 0 value then either infeasible or the BIG M was not big enough
        XB = 'infeasible or need bigger M'
        XN = 'infeasible or need bigger M'

  for IN_index in range(IN.size-1, -1, -1):                 #removing the artificial variables and there indeces from IN and XN
    if int(IN[:,IN_index]) > n:
      IN = np.delete(IN, IN_index, axis=1)
      XN = np.delete(XN, IN_index, axis=0)

  return XB,IB,XN,IN

In [ ]:
Simplex_generated_data = pd.DataFrame(columns=['Scenarios_no', 'Objective_function_value', 'Time_needed', 'Largest_percentage_change'])

for i_s in range(1, 602, 4):

  number_of_scenarios = i_s                                      #deciding the number of variables
  percentage = 0.0036                                            #deciding the percentage increment
  probabilities = 1/number_of_scenarios

  scenarios_list = np.array([])
  for scene in range(number_of_scenarios):
    scenarios_list = np.append(scenarios_list, scene+1)
  percentage_list = ((scenarios_list - np.median(scenarios_list))*percentage)+1                #getting the percentage list based on the number of scenarios

  #it was noticed that the A matrix followed a certain pattern when the scenarios increased. so it's been devided into 4 parts:
  #the initial constraint part (x1+x2+x3+s1 = 500), The coefficient of land acres which will be multiplied with the percentage, other variables coefficients which stays the same and form a super diagonal matrix, the rest of the slacks coefficients which also form a super diagonal matrix
  A_X_normal = np.array([[1, 1, 1, 1]])
  A_X = np.array([[2.5, 0, 0, 0],
                  [0, 3, 0, 0],
                  [0, 0, -20, 0],
                  [0, 0, 0, 0]])
  A_Main_rest = np.array([[1, 0, -1, 0, 0, 0],
                          [0, 1, 0, -1, 0, 0],
                          [0, 0, 0, 0, 1, 1],
                          [0, 0, 0, 0, 1, 0]])
  A_Slacks_rest = np.array([[-1, 0, 0, 0],
                            [0, -1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
  b_X = np.array([[500]])                                     #the initial constraint b (x1+x2+x3+s1 = 500), is not repeated
  b_Rest = np.array([[200],[240],[0],[6000]])                 #rest of b, repeated as many scenarios
  c_X = np.array([[150],[230],[260], [0]])                    #land acres c, are not repeated
  c_Rest = np.array([[238],[210],[-170],[-150],[-36],[-10]])                        #other variables c, repeated as many scenarios
  c_Slacks_rest = np.array([[0],[0],[0],[0]])                 #slacks c, repeated as many scenarios

  A_Main_rest_new = np.kron(np.eye(number_of_scenarios), A_Main_rest)               #putting this part of the A matrix into the super diagonal matrix form
  A_Main_rest_new[np.isclose(A_Main_rest_new, 0, atol=1e-10)] = 0                   #removing any small values generated instead of 0
  A_Slacks_rest_new = np.kron(np.eye(number_of_scenarios), A_Slacks_rest)
  A_Slacks_rest_new[np.isclose(A_Slacks_rest_new, 0, atol=9e-10)] = 0
  A_X_new = np.vstack([A_X * perc for perc in percentage_list])                     #multiplying this part with our percentage set
  A_Rest_new = np.hstack((A_X_new,A_Main_rest_new,A_Slacks_rest_new))               #stacking the three parts into on matrix
  num_columns = A_Rest_new.shape[1]
  zero_rows = np.zeros((1, num_columns - A_X_normal.shape[1]))
  A = np.vstack([np.hstack([A_X_normal, zero_rows]), A_Rest_new])                   #adding the initial constraint coefficient and getting our full A matrix

  b_Rest_new = np.vstack([b_Rest for num in range(number_of_scenarios)])            #multiplying the b part with as much constraint as we have then adding the constant part
  b = np.vstack([b_X,b_Rest_new])

  c_Rest_new = np.vstack([c_Rest * probabilities for num in range(number_of_scenarios)]) #multiplying the c part with as much constraint as we have then adding the constant part
  c_Slacks_rest_new = np.vstack([c_Slacks_rest for num in range(number_of_scenarios)])
  c = np.vstack((c_X,c_Rest_new,c_Slacks_rest_new))

  A = np.round(A, decimals=8)                                                       #rounding the values to eliminate any problem with recurring or decimal values
  A[np.isclose(A, 0, atol=1e-8)] = 0
  c = np.round(c, decimals=8)
  c[np.isclose(c, 0, atol=1e-8)] = 0

  start_time = time.time()

  XB, IB, XN, IN = BIG_M(A,b,c)                                                     #calling our BIGM function which also solves the LP
  total_index = np.hstack((IB,IN))                                                  #stacking the indices
  total_X = np.vstack((XB,XN))                                                      #stacking the variables
  total_index_flat = total_index.flatten()
  sorting_index = np.argsort(total_index_flat)
  X = total_X[sorting_index]                                                        #sorting the ivariables based on their indeces
  Objective_value = c.T@X                                                           #getting the objective function value

  end_time = time.time()
  elapsed_time = end_time - start_time

  Simplex_generated_data = Simplex_generated_data.append({'Scenarios_no': i_s, 'Objective_function_value': Objective_value.item(), 'Time_needed': elapsed_time, 'Largest_percentage_change': ((i_s-1)/2)*percentage}, ignore_index=True)

In [ ]:
Simplex_generated_data.to_excel('output.xlsx', index=False, header=False) #the time to go through the scenarios kept increasing, so they've been done in batches, each batch stored in an excel file, to be latered combined and demonstrated